# Valuing Interest Rate Derivatives. 
## Lattice model for Options
<p>Valuing a Callable Bond & Visualizing its price using the binomial option pricing model.</p>

<p>Binomial Models of Interest Rates. Binomial models are often used for computation of fixed income securities and derivatives without assuming log normality.This is an import implication as it allows us to price American style options. Traditional contniues time models like Black–Scholes can only price European options.  </p>

In [16]:
# Libraries we use 
suppressPackageStartupMessages(library(jrvFinance))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(optimx))
suppressPackageStartupMessages(library(nloptr))
suppressPackageStartupMessages(library(quantmod))
suppressPackageStartupMessages(library(lubridate))

In [17]:
# Functions we use 
# define DATE() function
DATE <- function(yyyy,mm,dd) {
  dte  <- as.Date(sprintf("%i-%i-%i",yyyy,mm,dd),format="%Y-%m-%d")
  return(dte)
}

# Defining a function to print binomial trees 
prt.tree <- function(tree,digit=2) {
  nt <- nrow(tree)
  # transpose tree
  trantree <- t(tree)
  nt1 <- 2*nt-1
  bintree  <- matrix(rep("",nt1*nt),nrow=nt1,ncol=nt)
  # convert to bin tree
  i1 <- nt
  for (j in 1:nt) {
    i1 <- nt-j+1
    for (i in 1:j) {
      bintree[i1,j] <- as.character(round(trantree[i,j],digit))
      i1 <- i1 + 2
    }
  }
  rownames(bintree) <- rep("",nt1)
  colnames(bintree) <- rep("",nt)
  return(noquote(bintree))
}

In [18]:
libor <- fread('/Users/robertgitari/Desktop/Fuqua\ Academics/Term4/Fixed\ Income\ Securities/Assignments/Team/3/Q1a.csv')
edf <- fread('/Users/robertgitari/Desktop/Fuqua\ Academics/Term4/Fixed\ Income\ Securities/Assignments/Team/3/Q1b.csv')

head(libor, 5)   # Libor Rates
head(edf, 5)     # Future Prices and their maturity 

term,LIBOR
Overnight,1.54263
1 week,1.63000
1 month,1.76250
2 month,1.83313
3 month,1.90838


maturity,FutPr
1/13/2020,98.1825
2/18/2020,98.2450
3/16/2020,98.2650
4/9/2020,98.2850
5/18/2020,98.3050


In [19]:
# number of compounding periods in a year
m = 2 

# length of a time step (in years)
delt = 0.5 

# number of periods 
N = 10 

# number of time steps; 10*0.5 = 5 YEAR BOND 
deltm  <- delt*m   

qtree= matrix(0, nrow=N+1, ncol=N+1)
for (i in 1:(N+1)) {
  for (j in 1:i) {
    qtree[i,j] = 0.5
  }
}

ztree = matrix(0, nrow=N+1, ncol=N+1)
ztree[1,1] = 0.1
for (i in 2:(N+1)) {
  ztree[i,1] = ztree[i-1,1]+0.01
  for (j in 2:i) {
    ztree[i,j] = ztree[i,j-1]-0.02
  }
}


## One-period spot rate Tree/Lattice:

In [20]:
prt.tree(ztree)

                                                      
                                                  0.2 
                                             0.19     
                                        0.18      0.18
                                   0.17      0.17     
                              0.16      0.16      0.16
                         0.15      0.15      0.15     
                    0.14      0.14      0.14      0.14
               0.13      0.13      0.13      0.13     
          0.12      0.12      0.12      0.12      0.12
     0.11      0.11      0.11      0.11      0.11     
 0.1      0.1       0.1       0.1       0.1       0.1 
     0.09      0.09      0.09      0.09      0.09     
          0.08      0.08      0.08      0.08      0.08
               0.07      0.07      0.07      0.07     
                    0.06      0.06      0.06      0.06
                         0.05      0.05      0.05     
                              0.04      0.04      0.04
          

In [21]:
# the price tree for the (non-callable) bond
# 10-period 10% coupon bond price tree
citree = matrix(0,nrow=N+1, ncol=N+1)
C = 5
for (i in 2:(N+1)) {
  citree[i,c(1:i)] <- rep(C,i)
}

ptree = matrix(0,nrow=N+1, ncol=N+1)   # price tree, including coupon
ptree[N+1,c(1:(N+1))] = rep(100,(N+1))
for (i in N:1) {
  i1 = i+1
  ptree[i,1:i] = (qtree[i,1:i]*(ptree[i+1,1:i]+citree[i+1,1:i])+
                    (1-qtree[i,1:i])*(ptree[i+1,2:i1]+citree[i+1,2:i1]))/(1+ztree[i,1:i]/m)^deltm
}


## Non-callable coupon bond price Tree/Lattice:

In [22]:
prt.tree(ptree)

                                                                          
                                                                       100
                                                                95.89     
                                                         92.97         100
                                                  91.07         96.77     
                                           90.08         94.65         100
                                    89.92         93.51         97.67     
                             90.52         93.24         96.39         100
                      91.86         93.8          96.04         98.59     
               93.92         95.14         96.55         98.17         100
        96.72         97.26         97.9          98.66         99.53     
 100.27        100.15        100.07        100.02        100           100
        103.85        103.05        102.24        101.38        100.48    
               106.89    

In [23]:
## Modify tree part C
#the price tree for the European call option on this bond, with the strike price of 100 and maturity of 3 years
X <- 100                                # strike price
Nopt <- 6                              # option maturity is 6 periods
ECall = matrix(0,nrow=Nopt+1, ncol=Nopt+1)   # option price tree
for (j in (1:(Nopt+1))) {
  ECall[Nopt+1,j] <- max( 0,ptree[Nopt+1,j]-X )
}
for (i in Nopt:1) {
  
  i1 = i+1
  ECall[i,1:i] = (qtree[i,1:i]*ECall[i+1,1:i]+(1-qtree[i,1:i])*ECall[i+1,2:i1])/(1+ztree[i,1:i]/m)^deltm
}

## European call option price

In [24]:
prt.tree(ECall,2)

                                    
                               0    
                          0         
                     0         0    
                0         0         
           0.19      0         0    
      0.64      0.4       0.01      
 1.35      1.16      0.84      0.02 
      2.19      2.04      1.76      
           3.42      3.42      3.65 
                5.08      5.37      
                     7.08      7.46 
                          9.22      
                               11.45

In [25]:
#The callable bond is the non-callable bond minus the European call option

for(i in 1:7){
  for(j in 1:7){
    ptree[i,j]<-ptree[i,j]-ECall[i,j]
  }
}


## Callable bond price tree
This callable bond price is just  the difference between the non-callable bond price tree and the European call option price tree

In [26]:
prt.tree(ptree)

                                                                        
                                                                     100
                                                              95.89     
                                                       92.97         100
                                                91.07         96.77     
                                          90.08        94.65         100
                                   89.92        93.51         97.67     
                            90.52         93.24        96.39         100
                     91.86         93.8         96.04         98.59     
              93.73         95.14         96.55        98.17         100
       96.07         96.85         97.89        98.66         99.53     
 98.92        98.98         99.22         100          100           100
       101.65        101.01        100.48       101.38        100.48    
              103.47        101.89        100      